In [ ]:
!unzip '/content/aclImdb.zip'

Streaming output truncated to the last 5000 lines.
  inflating: aclImdb/train/unsup/5504_0.txt  
  inflating: aclImdb/train/unsup/5505_0.txt  
  inflating: aclImdb/train/unsup/5506_0.txt  
  inflating: aclImdb/train/unsup/5507_0.txt  
  inflating: aclImdb/train/unsup/5508_0.txt  
  inflating: aclImdb/train/unsup/5509_0.txt  
  inflating: aclImdb/train/unsup/551_0.txt  
  inflating: aclImdb/train/unsup/5510_0.txt  
  inflating: aclImdb/train/unsup/5511_0.txt  
  inflating: aclImdb/train/unsup/5512_0.txt  
  inflating: aclImdb/train/unsup/5513_0.txt  
  inflating: aclImdb/train/unsup/5514_0.txt  
  inflating: aclImdb/train/unsup/5515_0.txt  
  inflating: aclImdb/train/unsup/5516_0.txt  
  inflating: aclImdb/train/unsup/5517_0.txt  
  inflating: aclImdb/train/unsup/5518_0.txt  
  inflating: aclImdb/train/unsup/5519_0.txt  
  inflating: aclImdb/train/unsup/552_0.txt  
  inflating: aclImdb/train/unsup/5520_0.txt  
  inflating: aclImdb/train/unsup/5521_0.txt  
  inflating: aclImdb/train/unsu

In [ ]:
#Import Necessary Libraries
import numpy as np
import pandas as pd
import nltk
import transformers
import tensorflow as tf
import keras
import os
import spacy
import re
import string
import bs4
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline
import warnings
warnings.simplefilter('ignore')

In [ ]:
#Read the file
def load_train_test_imdb(data_dir):
  data={}
  for split in ['test','train']:
    data[split]=[]
    for sentiment in ['neg','pos']:
      score=1 if sentiment=='pos' else 0
      path=os.path.join(data_dir,split,sentiment)
      filename=os.listdir(path)
      for f_name in filename[:150]:
        with open(os.path.join(path,f_name),'r',encoding='utf-8') as f:
          review=f.read()
          data[split].append([review,score])

  np.random.shuffle(data['train'])
  data['train']=pd.DataFrame(data['train'],columns=['Text','Sentiment'])

  np.random.shuffle(data['test'])
  data['test']=pd.DataFrame(data['test'],columns=['Text','Sentiment'])

  return data['train'],data['test']

In [ ]:
train_data,test_data=load_train_test_imdb(r'/content/aclImdb')

In [ ]:
#train data
train_data.head()

,Text,Sentiment
0,I saw this movie not knowing anything about it...,0
1,Sigh I sincerely wonder why all the acclaimed...,1
2,"On the surface, ""Show Me The Money"" should hav...",0
3,"You know all those letters to ""Father Christma...",0
4,For movie fans who have never heard of the boo...,0


In [ ]:
#test data
test_data.head()

,Text,Sentiment
0,"""Cover Girl"" is a lacklustre WWII musical with...",0
1,This film revolves around an Arabian leader (A...,1
2,Once upon a time Quentin Tarantino was a relat...,0
3,I can find no redeeming value to this movie. I...,0
4,Duchess and her three kittens are enjoying the...,1


In [ ]:
test_data.Text.values

array(['"Cover Girl" is a lacklustre WWII musical with absolutely nothing memorable about it, save for its signature song, "Long Ago and Far Away." This film came out before Gene Kelly really hit his artistic stride, and while there are evidences of his burgeoning talent here, mostly he plays sidekick to Rita Hayworth. And there\'s the problem. Rita Hayworth is gorgeous, no doubt about that. But she\'s simply not a compelling screen presence. I\'ve always found myself wanting to like her more than I actually do, and this movie is no exception. She\'s simply not a very good actress, and she\'s not even a very good dancer. Good looking as she is, there\'s something vapid about her, and this movie suffers because of it.<br /><br />Grade: C-',
       'This film revolves around an Arabian leader (Amir) who dies and wants to live on. So a Dr. Lloyd Trenton is being paid to transplant Amirs brain into a "willing" participant. But in the Doctors basement his dwarf assistant Dorro (Angelo Rossi

In [ ]:
train_data.Text.values

array(['Antonioni\'s movies have aged not well. What always surprised me about them is that, besides an unquestionable plastic beauty, there is a dull and didactic "psychology" of the characters and situations. Remember, for instance, the conversations between Mastroianni and the "wicked capitalistic" that wants sing up him in "La notte", or Monica Vitti laughing at the peasants flirting in the train in "La aventura", or Ferzetti dropping the glass of ink at the end of the same film. <br /><br />I have reviewed yesterday "Zabriskie Point". In this film there are a lot of nice and elaborate shots of the Rod Taylor office, the streets and highways of L.A., the publicity advertisements, the deserts,etc., that show the fascination of the author in his American journey, in the same way than Wim Wenders years later. Unfortunately, there are too a lot of hippie-leftist clichés that spoil the movie: - The boy leaves the meeting, steals an aeroplane and flies over the desert in order to liberat

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.



We will perform the below preprocessing tasks for our data:

1.Convert everything to lowercase

2.Remove HTML tags

3.Contraction mapping

4.Remove (‘s)

5.Remove any text inside the parenthesis ( )

6.Eliminate punctuations and special characters

7.Remove stopwords

8.Remove short words

Let’s define the function:

In [ ]:
def clean_text(text):
  text=text.lower()
  text=BeautifulSoup(text,'lxml').text
  text=re.sub(r'\([^)]*\)','',text)
  text=re.sub(r'[%s]'%re.escape(string.punctuation),'',text)
  text=re.sub(r'\w*\d\w*','',text)
  text=re.sub(r"'s\b",'',text)
  text=re.sub(r'[^a-zA-Z]',' ',text)
  if len(text)>1:
    return ' '.join(w for w in text.split() if w not in STOPWORDS)


In [ ]:
train_data['Clean_Text']=train_data['Text'].apply(lambda x:clean_text(x))

In [ ]:
#cleaned Text
train_data.head()

,Text,Sentiment,Clean_Text
0,Antonioni's movies have aged not well. What al...,0,antonionis movies aged well always surprised b...
1,Film makeovers of old TV shows seems to be the...,0,film makeovers old tv shows seems norm hollywo...
2,"I never wanted to see this film, then one day,...",0,never wanted see film one day joke watched see...
3,I'm a HUGE fan of the twin sisters. Although t...,1,im huge fan twin sisters although one soo good...
4,"Not sure why it doesn't play in Peoria, appare...",1,sure doesnt play peoria apparently funny cleve...


In [ ]:
train_data.Clean_Text.values

array(['antonionis movies aged well always surprised besides unquestionable plastic beauty dull didactic psychology characters situations remember instance conversations mastroianni wicked capitalistic wants sing la notte monica vitti laughing peasants flirting train la aventura ferzetti dropping glass ink end film reviewed yesterday zabriskie point film lot nice elaborate shots rod taylor office streets highways la publicity advertisements desertsetc show fascination author american journey way wim wenders years later unfortunately lot hippieleftist clich spoil movie boy leaves meeting steals aeroplane flies desert order liberate find something different executives grey suites speak time speculation girl looks object women swimming pool leaves wants like couple fat middleclass caravan speak front beauty nature building hotel earning lot money last least lot couples making love desert hippie platitudesorry today half century revolution la aventura see king naked films handful aesthetic

In [ ]:
train_data.drop('Text',axis=1,inplace=True)

In [ ]:
#Performing Lemmatization
nlp=spacy.load('en_core_web_sm',disable=['ner','parser'])

def lemmatizer(text):
  sent=[]
  doc=nlp(text)
  for word in doc:
    sent.append(word.lemma_)
  return ' '.join(sent)

train_data['Lemmatize_Text']=train_data['Clean_Text'].apply(lambda x:lemmatizer(x))
train_data['Lemmatize_Text_Clean']=train_data['Lemmatize_Text'].str.replace('-PRON-','')

In [ ]:
train_data.head()

,Sentiment,Clean_Text,Lemmatize_Text,Lemmatize_Text_Clean
0,0,antonionis movies aged well always surprised b...,antonionis movie age well always surprised bes...,antonionis movie age well always surprised bes...
1,0,film makeovers old tv shows seems norm hollywo...,film makeover old tv show seem norm hollywood ...,film makeover old tv show seem norm hollywood ...
2,0,never wanted see film one day joke watched see...,never want see film one day joke watch see bad...,never want see film one day joke watch see bad...
3,1,im huge fan twin sisters although one soo good...,I m huge fan twin sister although one soo good...,I m huge fan twin sister although one soo good...
4,1,sure doesnt play peoria apparently funny cleve...,sure do not play peoria apparently funny cleve...,sure do not play peoria apparently funny cleve...


In [ ]:
train_data=train_data.iloc[:,[1,2,3,0]]
train_data.head()

,Clean_Text,Lemmatize_Text,Lemmatize_Text_Clean,Sentiment
0,antonionis movies aged well always surprised b...,antonionis movie age well always surprised bes...,antonionis movie age well always surprised bes...,0
1,film makeovers old tv shows seems norm hollywo...,film makeover old tv show seem norm hollywood ...,film makeover old tv show seem norm hollywood ...,0
2,never wanted see film one day joke watched see...,never want see film one day joke watch see bad...,never want see film one day joke watch see bad...,0
3,im huge fan twin sisters although one soo good...,I m huge fan twin sister although one soo good...,I m huge fan twin sister although one soo good...,1
4,sure doesnt play peoria apparently funny cleve...,sure do not play peoria apparently funny cleve...,sure do not play peoria apparently funny cleve...,1


In [ ]:
#cleaning the text data
test_data['Clean_Text']=test_data['Text'].apply(lambda x:clean_text(x))
test_data['Lemmatize_Text']=test_data['Clean_Text'].apply(lambda x:lemmatizer(x))
test_data['Lemmatize_Text_Clean']=test_data['Lemmatize_Text'].str.replace('-PRON-','')

In [ ]:
test_data.drop('Text',axis=1,inplace=True)

In [ ]:
test_data.head()

,Sentiment,Clean_Text,Lemmatize_Text,Lemmatize_Text_Clean
0,1,searched movie years great memories firstrate ...,search movie year great memory firstrate actin...,search movie year great memory firstrate actin...
1,0,wife kids billed cosby show unlike latter unfu...,wife kid bill cosby show unlike latter unfunny...,wife kid bill cosby show unlike latter unfunny...
2,0,spoiler needed steer clear thiswell bizarre fi...,spoiler need steer clear thiswell bizarre film...,spoiler need steer clear thiswell bizarre film...
3,0,derivative erotic thriller remains watchable w...,derivative erotic thriller remain watchable wa...,derivative erotic thriller remain watchable wa...
4,1,minutes song scene daryl hannah jfk jr first e...,minute song scene daryl hannah jfk jr first em...,minute song scene daryl hannah jfk jr first em...


In [ ]:
test_data=test_data.iloc[:,[1,2,3,0]]
test_data.head()

,Clean_Text,Lemmatize_Text,Lemmatize_Text_Clean,Sentiment
0,searched movie years great memories firstrate ...,search movie year great memory firstrate actin...,search movie year great memory firstrate actin...,1
1,wife kids billed cosby show unlike latter unfu...,wife kid bill cosby show unlike latter unfunny...,wife kid bill cosby show unlike latter unfunny...,0
2,spoiler needed steer clear thiswell bizarre fi...,spoiler need steer clear thiswell bizarre film...,spoiler need steer clear thiswell bizarre film...,0
3,derivative erotic thriller remains watchable w...,derivative erotic thriller remain watchable wa...,derivative erotic thriller remain watchable wa...,0
4,minutes song scene daryl hannah jfk jr first e...,minute song scene daryl hannah jfk jr first em...,minute song scene daryl hannah jfk jr first em...,1


In [ ]:
from transformers import BertTokenizer,TFBertForSequenceClassification
from transformers import InputExample,InputFeatures

In [ ]:
#Bert Tokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
#Bert model
model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model.summary())

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
#convert the data to examples
def convert_data_to_examples(train_data,test_data,TEXT,SENTIMENT):
  train_InputExamples=train_data.apply(lambda x:InputExample(guid=None,
                                                             text_a=x[TEXT],
                                                             text_b=None,
                                                             label=x[SENTIMENT]),axis=1)

  test_InputExamples=test_data.apply(lambda x:InputExample(guid=None,
                                                           text_a=x[TEXT],
                                                           text_b=None,
                                                           label=x[SENTIMENT]),axis=1)

  return train_InputExamples,test_InputExamples

In [ ]:
#preprocessing the model
def convert_examples_to_tf_dataset(examples,tokenizer,maxlength=128):
    features=[]
    for e in examples:
        Input_dict=tokenizer.encode_plus(
          text=e.text_a,
          add_special_tokens=True,
          max_length=maxlength,
          pad_to_max_length=True,
          truncation=True,
          return_token_type_ids=True,
          return_attention_mask=True)

        input_ids,token_type_ids,attention_mask=(Input_dict['input_ids'],
             Input_dict['token_type_ids'],Input_dict['attention_mask'])

        features.append(
           InputFeatures(
           input_ids=input_ids,token_type_ids=token_type_ids,attention_mask=attention_mask,label=e.label))

    def gen():
        for f in features:
            yield(
                 {'input_ids':f.input_ids,
                  'token_type_ids':f.token_type_ids,
                  'attention_mask':f.attention_mask},
                   f.label,)

    return tf.data.Dataset.from_generator(
     gen,
     ({'input_ids':tf.int32,'token_type_ids':tf.int32,'attention_mask':tf.int32},tf.int64),
     ({'input_ids':tf.TensorShape([None]),
       'token_type_ids':tf.TensorShape([None]),
       'attention_mask':tf.TensorShape([None])},
       tf.TensorShape([]),),)


In [ ]:
TEXT='Lemmatize_Text_Clean'
SENTIMENT='Sentiment'

In [ ]:
train_InputExamples,test_InputExamples=convert_data_to_examples(train_data,test_data,TEXT,SENTIMENT)

In [ ]:
from random import shuffle
train_data=convert_examples_to_tf_dataset(list(train_InputExamples),tokenizer)
train_data=train_data.shuffle(100).batch(32).repeat(2)

test_data=convert_examples_to_tf_dataset(list(test_InputExamples),tokenizer)
test_data=test_data.batch(32)

In [ ]:
#optimizer and the loss-function
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5,epsilon=1e-8,clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(train_data,epochs=20,validation_data=test_data)

Epoch 1/20
20/20 [==============================] - 86s 1s/step - loss: 0.6843 - accuracy: 0.5700 - val_loss: 0.6698 - val_accuracy: 0.5800
Epoch 2/20
20/20 [==============================] - 20s 1s/step - loss: 0.4820 - accuracy: 0.7967 - val_loss: 0.7655 - val_accuracy: 0.5933
Epoch 3/20
20/20 [==============================] - 21s 1s/step - loss: 0.2685 - accuracy: 0.9017 - val_loss: 0.9408 - val_accuracy: 0.6567
Epoch 4/20
20/20 [==============================] - 19s 947ms/step - loss: 0.1228 - accuracy: 0.9550 - val_loss: 1.0136 - val_accuracy: 0.6600
Epoch 5/20
20/20 [==============================] - 19s 938ms/step - loss: 0.0283 - accuracy: 0.9967 - val_loss: 1.0912 - val_accuracy: 0.7033
Epoch 6/20
20/20 [==============================] - 19s 951ms/step - loss: 0.0158 - accuracy: 0.9983 - val_loss: 1.2115 - val_accuracy: 0.7067
Epoch 7/20
20/20 [==============================] - 19s 960ms/step - loss: 0.0315 - accuracy: 0.9917 - val_loss: 1.2305 - val_accuracy: 0.7000
Epoch 8/

In [ ]:
#Evaluate the model
loss,acc=model.evaluate(test_data)
print('Test Accuracy:%f'%(acc*100))

10/10 [==============================] - 3s 286ms/step - loss: 2.2829 - accuracy: 0.6433
Test Accuracy:64.333332


In [ ]:
pred_sentences = ['This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good',
                  'One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie']

In [ ]:
#prediction
tf_batch=tokenizer(pred_sentences,max_length=128,padding=True,truncation=True,return_tensors='tf')
tf_output=model(tf_batch)
tf_predictions=tf.nn.softmax(tf_output[0],axis=-1)
labels=['Negative','Positive']
label=tf.argmax(tf_predictions,axis=1)
label=label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i],':\n',labels[label[i]])

This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good :
 Positive
One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie :
 Negative
